## Load configurations

In [1]:
from cfgs.content_classification_v1 import get_cfg

cfgs = get_cfg()
print('\n', cfgs, '\n')


 CLASSES: [{'Genre': {0: 'Comedy', 1: 'Drama', 2: 'Action', 3: 'Documentary', 4: 'Crime', 5: 'Animation', 6: 'Horror', 7: 'Adventure'}, 'Rating': {0: 'value'}, 'Year': {0: 'value'}}]
HYPERPARAMS:
  EPOCHS: 100
  LR: 0.0001
  TEST_SPLIT: 0.8
  VAL_SPLIT: 0.8
IMAGE:
  RESOLUTION: (224, 224, 3)
MODEL:
  BACKBONE: InceptionV3
  STRUCTURE: [{0: {'type': 'Genre', 'outNeurons': 3, 'outActivation': 'sigmoid', 'loss': 'categorical_crossentropy', 'weight': 1, 'metric': 'accuracy'}, 1: {'type': 'Rating', 'outNeurons': 1, 'outActivation': 'linear', 'loss': 'mse', 'weight': 1, 'metric': 'mae'}, 2: {'type': 'Year', 'outNeurons': 1, 'outActivation': 'linear', 'loss': 'mse', 'weight': 1, 'metric': 'mae'}}] 



In [2]:
dataset_dict = cfgs.CLASSES[0]
dataset_dict['Genre_alias'] = dict((g, i) for i, g in dataset_dict['Genre'].items())

## Load data and initialize generator

In [3]:
import pandas as pd
from data_processing.data_generator import DataGenerator

In [4]:
df = pd.read_pickle("/home/robotics/content-classification/data_processing/balanced_data.pkl")

In [5]:
data_generator = DataGenerator(df, dataset_dict, cfgs.HYPERPARAMS.TEST_SPLIT, cfgs.HYPERPARAMS.VAL_SPLIT, cfgs.IMAGE.RESOLUTION)
data_generator.df.head()

,Index,Name,Year,Genre,Rating,Poster_path
0,tt0110901,Project Shadowchaser II,1995,2,4.0,/home/robotics/Documents/data/Posters/1995/tt0...
1,tt0365265,Ginger Snaps Back: The Beginning,2004,1,5.9,/home/robotics/Documents/data/Posters/2004/tt0...
2,tt0244992,Barrela: Escola de Crimes,1990,1,6.5,/home/robotics/Documents/data/Posters/1990/tt0...
3,tt1648188,Sharunas Bartas: An Army of One,2010,3,6.5,/home/robotics/Documents/data/Posters/2010/tt1...
4,tt1935737,Aquatic Language,2002,5,6.3,/home/robotics/Documents/data/Posters/2002/tt1...


In [10]:
train_idx, valid_idx, test_idx = data_generator.split_dataset() 

## Build and compile model

In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from model.architecture import MultiOutputModel

In [8]:
structure = cfgs.MODEL.STRUCTURE[0]
content_model = MultiOutputModel(*cfgs.IMAGE.RESOLUTION, structure)
content_model.build_model("InceptionV3")
# keras.utils.plot_model(content_model.model, show_shapes=True)

In [9]:
opt = Adam(lr=HYPERPARAMS.LR, decay=HYPERPARAMS.LR/HYPERPARAMS.EPOCHS)
content_model.compile_model(opt)

NameError: name 'HYPERPARAMS' is not defined